##### Step one: prepare the data and churn rate

In [ ]:
df_obs = df.loc[(df['Purchase_dt'] >= '2010-02-01') & (df['Purchase_dt'] <= '2011-01-31')]
cus_list=df_obs['CustomerID'].unique()

df_post = df.loc[df['Purchase_dt']>'2011-01-31']
cus_after_list = df_post['CustomerID'].unique()

num_left = cus_list[np.isin(cus_list, cus_after_list, invert = True)]
churn_rate = num_left.shape[0] / cus_list.shape[0]

##### Step Two: calculate R, F , M score，and total RFM score

In [ ]:
df_rfm['current'] = pd.to_datetime('2011-02-01')
df_rfm['range']=df_rfm['current']-df_rfm['Purchase_dt']

In [ ]:
df_rfm.groupby('CustomerID')['Invoice'].nunique()

In [ ]:
df_rfm['TotalPrice'] = df_rfm['Quantity'] * df_rfm['Price']

In [ ]:
rfm = df_rfm.groupby('CustomerID').agg(recency=pd.NamedAgg(column='range', aggfunc='min'),
                                                  frequency=pd.NamedAgg(column='Invoice', aggfunc='nunique'),
                                                  monetary=pd.NamedAgg(column='TotalPrice', aggfunc='sum')
                                                 )

- rank by each RFM value and generate the score:

In [ ]:
rfm['r_percentile'] = rfm['recency'].rank(method='first',pct=True,ascending=True)
rfm['r_score'] = pd.qcut(rfm['r_percentile'], q=4, labels=range(4,0,-1))

rfm['f_percentile'] = rfm['frequency'].rank(method='first',pct=True,ascending=False)
rfm['f_score'] = pd.qcut(rfm['f_percentile'],q=4,labels=range(4,0,-1))

rfm['m_percentile'] = rfm['monetary'].rank(method='first',pct=True,ascending=False)
rfm['m_score'] = pd.qcut(rfm['m_percentile'],4,range(4,0,-1))

rfm['rfm_score'] = rfm['r_score'] + rfm['f_score'] + rfm['m_score']

##### Step Three：Calculate the average CLV for each RFM segment

In [ ]:
clv=rfm.groupby(['rfm_score']).agg(total_customers=pd.NamedAgg(column='CustomerID', aggfunc='nunique'),
                                                  monetary=pd.NamedAgg(column='monetary', aggfunc='sum'))

clv['clv']=(clv['monetary']/clv['total_customers'])*1/churn_rate